本段尽可能通过调用api的方式进行。

In [ ]:
import pyaudio,wave
import ollama
import warnings
import os
import time
import glob
import sys
import torch
import re
import logging
import pickle
os.environ['HF_ENDPOINT']='hf-mirror.com'#hugging_face镜像
# os.environ['HF_HUB_OFFLINE']='0'
#清理内存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
if hasattr(torch.cuda, 'empty_cache'):
    torch.cuda.empty_cache()
del torch
warnings.filterwarnings("ignore")

original_path = sys.path

In [ ]:
print(os.getcwd())
sys.path

In [ ]:
is_voice=True#是否是语音输入
sherpa=False

In [ ]:
#语音合成：GPT_Sovits---fast_inference version
from GPT_SoVITS.inference_webui_fast_inference_maple import handle

#获取LargeModel所在的路径（事先将它添加到PYTHONPATH的环境变量中）
path= [content for content in sys.path if re.match('.+:.*LargeModel$', content)][0]
#ASR模型(Automatic Speech Recognition)
if is_voice:
    if sherpa:
        #ASR模型：sherpa_onnx---cpu版本
        import sherpa_onnx
        from kaldi.sherpa_onnx_model.stream_zipformer import microphone_endpoint
        #获取stream_zipformer所在的文件路径
        sherpa_path = path+'/'+'kaldi/sherpa_onnx_model/stream_zipformer'
        args_dict = {  
            'tokens': sherpa_path+'/tokens.txt',  
            'encoder': sherpa_path+'/encoder-epoch-99-avg-1.onnx',  
            'decoder': sherpa_path+'/decoder-epoch-99-avg-1.onnx' ,  
            'joiner': sherpa_path+'/joiner-epoch-99-avg-1.onnx',    
        } 
        #检查设备，顺便输出recongnizer
        check_recognizer= microphone_endpoint.main(args_dict)
        del sherpa_path
    else:
        from Speech_to_Text.Fast_whisper.Fast_whisper import fast_whisper_realtime
        from faster_whisper import WhisperModel
        model_size='large-v3'
        download_root=path+'/Speech_to_Text/fast_whisper'
        #当download_root与模型在同一个目录时候
        model = WhisperModel(download_root+'\\'+model_size, device="cuda",compute_type="int8_float16", download_root=download_root, local_files_only=False)


#文本模型：chatglm
from zhipuai import ZhipuAI
with open(path+'/Language_Model/Text_generation/chatglm_api/api_key.txt', 'r') as chatglm_api:
    api_key = chatglm_api.read()
client = ZhipuAI(api_key=api_key.split('=')[1].strip('"')) # 填写您自己的APIKey

In [ ]:
text1 = '''
当然，作为你的朋友，我们可以聊聊月亮、星星，或者任何你感兴趣的话题。月亮总是给人以浪漫和宁静的感觉，你有没有什么特别喜欢关于月亮的诗句或者故事想要分享的呢？
'''

In [ ]:
#语音合成模型初始化
ref_wav_path=r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\派蒙-效果并不好\参考音频\说话—既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。.wav"
ref_wav_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\面试官这边临时有改动，然后，明天下午两点可以吗？.wav"
ref_wav_path =r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\这是一个传承了千年尊贵的名字。.wav"
prompt_text= '既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。'
prompt_text = "面试官这边临时有改动，然后，明天下午两点可以吗？"
prompt_text = "这是一个传承了千年，尊贵的名字。"
text='晚上好，祝你有一个甜美的睡眠！'#'你好呀！'
# text=text1
top_k=5
top_p=1
temperature=1
how_to_cut='按标点符号切'
# how_to_cut='凑四句一切'
sovits_path = r"GPT_SoVITS\pretrained_models\s2G488k.pth"
# sovits_path=r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\xunfeih23r_denoise_e16_s800.pth"
sovits_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise_e25_s575.pth"
gpt_path = r"GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt"
# gpt_path = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\xunfeih23r\数据集\xunfeih23r_denoise-e15.ckpt"
gpt_path =r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise-e25.ckpt"
audio_save=False
audio_save_file='output.wav'
text_language2='中英混合'
prompt_language2='中英混合'
batch_size=8         # int. batch size for inference
split_bucket=True   # bool. whether to split the batch into multiple buckets.
speed_factor=1.0
ref_text_free=False

top_k=5
top_p=1
speed_factor=1.1

params={'text':text, 'text_lang':text_language2, 
            'ref_audio_path':ref_wav_path, 'prompt_text':prompt_text, 
            'prompt_lang':prompt_language2, 'top_k':top_k, 
            'top_p':top_p, 'temperature':temperature, 
            'text_split_method':how_to_cut, 'batch_size':batch_size, 
            'speed_factor':speed_factor, 'ref_text_free':ref_text_free,
            'split_bucket':split_bucket, 'audio_save':audio_save, 
            'audio_save_file':audio_save_file, 'mygpt_path':gpt_path, 'mysovits_path':sovits_path}

def tts(params):
    # 将标准输出流重定向到空设备
    original_stdout =  sys.stdout
    sys.stdout = open('log_maple.txt', 'w')
    handle(**params)
    sys.stdout = original_stdout
    logging.getLogger().setLevel(logging.WARNING)
tts(params)

In [ ]:
#拜拜结束对话
#对话前删除临时录音文件
def delete_files(folder_path):
    file_list = glob.glob(folder_path + '/*')
    for file_path in file_list:
        if os.path.isfile(file_path):
            if '.' in file_path and file_path.split('.')[0].split('/')[-1][:4]=='test':
                os.remove(file_path)

# 调用函数删除文件夹中的文件
audio_file_path = 'test'
delete_files(audio_file_path)

#语音提醒可以对话了
params['text'] ="您好，您现在可以跟我进行对话了！" 
tts(params)

outfile_name=audio_file_path+'\\test.mp3'
lc =0 #对话轮次，用于录音文件的名称存储
messages = [{"role": "system", "content":"您将以朋友的身份与用户进行聊天对话。"}]
tokens_use = 0

awake=False #表示是否是语音唤醒部分   
while(True):#语音对话的循环
    #ASR
    if is_voice:
        if sherpa:
            #last_result：一轮的最后一段结果, lc_result：一轮的结果
            last_result, lc_result = microphone_endpoint.main(args_dict, False, check_recognizer)
        else:
            delayTime=0.8
            tendure=3
            mindb=2000
            last_result, lc_result=fast_whisper_realtime(audio_file_path, model, delayTime, tendure, mindb)
        #语音唤醒
        if awake:
            if last_result in {"老铁你好","千龙你好","乾隆你好","青龙你好",
                            "天龙你好","钱老你好","前路你好","成龙你好"}:
                params['text']='我在呢！'
                tts(params)
                awake=False
            else:
                time.sleep(0.5)
            continue
        #结束语

        if re.sub(r'[ .。！!?？-]','', last_result).lower() in {'拜拜','byebye','再见','掰掰'}:
            lc=-1
            result=last_result
        
        if len(lc_result)==0:
            print('\n', '*'*40)
            print(' '*5, '您的对话已超时，请重新对话！')
            print('\n', '*'*40)
            awake=True
            continue

        print()
        if lc==-1:
            print(result)
            break
        lc +=1
        question = lc_result
        # if (len(question)==0)|(question=='拜拜'):
        #     break
    else:
        question = input('请输入你的问题：回车退出')
        if len(question)==0:
            lc=-1
            break
        print('用户：', end='')
        print(question)

    #--------------------Text to Text
    #剪切历史记录每9条删除前面4条
    if len(messages)>9:
        for i in range(4):
            messages.pop(1)
    print('智谱：', end='')
    messages.append({"role": "user", "content": question})
    llm_model = 'glm-4'
    response = client.chat.completions.create(
        model=llm_model,  # 填写需要调用的模型名称
        messages=messages,
        # max_tokens=1024,#所以历史记录加起来不能超过这个长度
        stream=True,
    )
    answer=''
    outword=''
    for chunk in response:
        answer_chunk = chunk.choices[0].delta.content
        temp_split_left = answer_chunk
        temp_split_right = ''
        #是否播放音频——初始为False
        is_tts=False
        #'^(.\n)|(^[。\n]).*'任意\n在前两位及开头是。
        if re.search('^(.\n)|(^[。\n]).*',answer_chunk):
            #智谱的\n可能在chunk的中间，把最后一个\n之前的视为上一句
            temp_split = re.split('\n+', answer_chunk)
            if len(temp_split)>1:
                temp_split_left = '\n'.join(temp_split[:-1])+'\n'
                temp_split_right = temp_split[-1]
            is_tts=True 
        outword += temp_split_left
        answer += temp_split_left
        print(temp_split_left, end='', flush=True)
        if is_tts:
            params['text']= outword
            tts(params)
            outword=temp_split_right
            answer += temp_split_right
            print(temp_split_right, end='', flush=True)
    if len(outword)!=0:
        params['text']=outword
        tts(params)
    print('##end', end='')
    params['text'] ='##end'
    tts(params)
    print()
    tokens_use += chunk.usage.total_tokens
    messages.append({"role": "assistant", "content": answer})

In [ ]:
#存储聊天记录
with open(r'E:\LargeModel\Language_Model\Text_generation\chatglm_api\chat_message.pkl', 'wb') as save_file:
    pickle.dump(messages, save_file)
tokens_use